# From relational to graph databases

## MySQL settings

I couldn't get MySQL to work... so I am moving to a pandas situation.

## Schema design

### Ingestion considerations

In [1]:
import pandas as pd

In [2]:
all_play = pd.read_csv('data/steam_play.csv', header = None)
all_play.columns = ['id', 'game_name', 'type', 'hours', 'misc_column']
all_play.id = all_play.id.astype(str)
all_play

,id,game_name,type,hours,misc_column
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,play,87.0,0
2,151603712,Spore,play,14.9,0
3,151603712,Fallout New Vegas,play,12.1,0
4,151603712,Left 4 Dead 2,play,8.9,0
...,...,...,...,...,...
70484,128470551,Fallen Earth,play,2.4,0
70485,128470551,Magic Duels,play,2.2,0
70486,128470551,Titan Souls,play,1.5,0
70487,128470551,Grand Theft Auto Vice City,play,1.5,0


In [3]:
play_data = all_play[['id', 'game_name', 'hours']]
print(play_data[:10])

          id                   game_name  hours
0  151603712  The Elder Scrolls V Skyrim  273.0
1  151603712                   Fallout 4   87.0
2  151603712                       Spore   14.9
3  151603712           Fallout New Vegas   12.1
4  151603712               Left 4 Dead 2    8.9
5  151603712                    HuniePop    8.5
6  151603712               Path of Exile    8.1
7  151603712                 Poly Bridge    7.5
8  151603712                 Left 4 Dead    3.3
9  151603712             Team Fortress 2    2.8


In [4]:
all_purch = pd.read_csv('data/steam_purchase.csv', header = None)
all_purch.columns = ['id', 'game_name', 'type', 'game_purchased_flag', 'misc_column']
all_purch.id = all_purch.id.astype(str)
all_purch

,id,game_name,type,game_purchased_flag,misc_column
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Spore,purchase,1.0,0
3,151603712,Fallout New Vegas,purchase,1.0,0
4,151603712,Left 4 Dead 2,purchase,1.0,0
...,...,...,...,...,...
129506,128470551,Fallen Earth,purchase,1.0,0
129507,128470551,Magic Duels,purchase,1.0,0
129508,128470551,Titan Souls,purchase,1.0,0
129509,128470551,Grand Theft Auto Vice City,purchase,1.0,0


In [5]:
purchase_data = all_purch[['id', 'game_name']]
print(purchase_data[:10])

          id                   game_name
0  151603712  The Elder Scrolls V Skyrim
1  151603712                   Fallout 4
2  151603712                       Spore
3  151603712           Fallout New Vegas
4  151603712               Left 4 Dead 2
5  151603712                    HuniePop
6  151603712               Path of Exile
7  151603712                 Poly Bridge
8  151603712                 Left 4 Dead
9  151603712             Team Fortress 2


In [6]:
users = set(play_data['id'].tolist() + purchase_data['id'].tolist())
user_ids = {user_id: igraph_id for igraph_id, user_id in enumerate(users)}
print(len(user_ids))

12393


In [7]:
games = set(play_data['game_name'].tolist() + purchase_data['game_name'].tolist())
game_ids = {user_id: igraph_id for igraph_id, user_id in enumerate(games, len(user_ids))}
print(len(game_ids))

5155


In [8]:
print(sorted(user_ids.values(), reverse=True)[:10])
print(sorted(game_ids.values(), reverse=False)[:10])

[12392, 12391, 12390, 12389, 12388, 12387, 12386, 12385, 12384, 12383]
[12393, 12394, 12395, 12396, 12397, 12398, 12399, 12400, 12401, 12402]


In [9]:
all_ids = sorted(list(user_ids.values()) + list(game_ids.values()))
assert all_ids == list(range(len(all_ids)))

Now that we have modified the data to be ready for the graph, let's create an empty graph.

In [10]:
import igraph as ig
g = ig.Graph(directed=True)

In [11]:
user_ids = dict(sorted(user_ids.items(), key=lambda item: item[1]))
game_ids = dict(sorted(game_ids.items(), key=lambda item: item[1]))

In [12]:
steam_user_ids = list(user_ids.keys())
steam_game_ids = list(game_ids.keys())

In [13]:
g.add_vertices(len(steam_user_ids) + len(steam_game_ids))
assert len(g.vs) == len(steam_user_ids) + len(steam_game_ids)

In [14]:
all_steam_ids = steam_user_ids + steam_game_ids
all_steam_ids

['20643309',
 '276073724',
 '285972122',
 '301099545',
 '239004331',
 '265832700',
 '176968027',
 '164772402',
 '219677174',
 '195105284',
 '79658056',
 '31069041',
 '33706322',
 '183086571',
 '189158075',
 '96634230',
 '202894728',
 '207510132',
 '303413869',
 '163043651',
 '248303087',
 '18066817',
 '146594813',
 '173967126',
 '255892249',
 '174674290',
 '264399888',
 '54507379',
 '298263185',
 '124234882',
 '10450544',
 '215667739',
 '123989127',
 '256212635',
 '296254406',
 '140050187',
 '159538705',
 '98221974',
 '202961177',
 '145132643',
 '161816929',
 '124429562',
 '182851135',
 '96515834',
 '165073598',
 '217770237',
 '27262175',
 '192080216',
 '7527129',
 '234950941',
 '257288675',
 '5250',
 '75988937',
 '33452361',
 '203729080',
 '145078067',
 '289302686',
 '242988245',
 '201176964',
 '114751336',
 '70381757',
 '218600642',
 '68240622',
 '309038666',
 '243587325',
 '244376049',
 '158277459',
 '124776658',
 '124059151',
 '182593878',
 '121770429',
 '242351732',
 '144845684',


In [15]:
node_types = ['user' for _ in steam_user_ids] + ['game' for _ in steam_game_ids]


Steam ID for each node is added as a property. 

Type for each node (game or user) is added as a property.

In [16]:
g.vs['steam_id'] = all_steam_ids
g.vs['type'] = node_types

In [17]:
print(g.vs['steam_id'][:10])
print(g.vs['type'][:10])

['20643309', '276073724', '285972122', '301099545', '239004331', '265832700', '176968027', '164772402', '219677174', '195105284']
['user', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'user']


In [18]:
game_nodes = g.vs.select(type_eq='game')
print(len(game_nodes))

5155


In [19]:
purchase_edges = [[user_ids[row.id], game_ids[row.game_name]] for row in purchase_data.itertuples()]
purchase_edges

[[1980, 13154],
 [1980, 14009],
 [1980, 15920],
 [1980, 16250],
 [1980, 17353],
 [1980, 14439],
 [1980, 14187],
 [1980, 12908],
 [1980, 14770],
 [1980, 17264],
 [1980, 14261],
 [1980, 14109],
 [1980, 14135],
 [1980, 13004],
 [1980, 17317],
 [1980, 13946],
 [1980, 13546],
 [1980, 14253],
 [1980, 15688],
 [1980, 16839],
 [1980, 14531],
 [1980, 17402],
 [1980, 17136],
 [1980, 14197],
 [1980, 12772],
 [1980, 13230],
 [1980, 16071],
 [1980, 15007],
 [1980, 15332],
 [1980, 16577],
 [1980, 13782],
 [1980, 14330],
 [1980, 16308],
 [1980, 14283],
 [1980, 16932],
 [1980, 16435],
 [1980, 15831],
 [1980, 16199],
 [1980, 16826],
 [1980, 14842],
 [5874, 17402],
 [5083, 17148],
 [5083, 12644],
 [5083, 13154],
 [5083, 15374],
 [5083, 14146],
 [5083, 14806],
 [5083, 14438],
 [5083, 17146],
 [5083, 16725],
 [5083, 14141],
 [5083, 14686],
 [5083, 13406],
 [5083, 17197],
 [5083, 12622],
 [5083, 16319],
 [5083, 17275],
 [5083, 15931],
 [5083, 16661],
 [5083, 16775],
 [5083, 15776],
 [5083, 13233],
 [5083, 

In [20]:
play_data

,id,game_name,hours
0,151603712,The Elder Scrolls V Skyrim,273.0
1,151603712,Fallout 4,87.0
2,151603712,Spore,14.9
3,151603712,Fallout New Vegas,12.1
4,151603712,Left 4 Dead 2,8.9
...,...,...,...
70484,128470551,Fallen Earth,2.4
70485,128470551,Magic Duels,2.2
70486,128470551,Titan Souls,1.5
70487,128470551,Grand Theft Auto Vice City,1.5


In [21]:
play_edges = [[user_ids[row.id], game_ids[row.game_name], row.hours] for row in play_data.itertuples()]
play_edges

[[1980, 13154, 273.0],
 [1980, 14009, 87.0],
 [1980, 15920, 14.9],
 [1980, 16250, 12.1],
 [1980, 17353, 8.9],
 [1980, 14439, 8.5],
 [1980, 14187, 8.1],
 [1980, 12908, 7.5],
 [1980, 14770, 3.3],
 [1980, 17264, 2.8],
 [1980, 14261, 2.5],
 [1980, 14109, 2.0],
 [1980, 14135, 1.4],
 [1980, 13004, 1.3],
 [1980, 17317, 1.3],
 [1980, 13946, 0.8],
 [1980, 13546, 0.8],
 [1980, 14253, 0.6],
 [1980, 15688, 0.5],
 [1980, 16839, 0.5],
 [1980, 14531, 0.5],
 [1980, 17402, 0.5],
 [1980, 17136, 0.5],
 [1980, 14197, 0.4],
 [1980, 12772, 0.1],
 [1980, 13230, 0.1],
 [5874, 17402, 2.3],
 [5083, 17148, 238.0],
 [5083, 12644, 84.0],
 [5083, 13154, 58.0],
 [5083, 15374, 22.0],
 [5083, 14146, 13.8],
 [5083, 14806, 10.2],
 [5083, 14438, 7.8],
 [5083, 17146, 7.0],
 [5083, 16725, 5.2],
 [5083, 14141, 4.8],
 [5083, 14686, 3.4],
 [5083, 13406, 3.2],
 [5083, 17197, 2.8],
 [5083, 12622, 2.6],
 [5083, 16319, 2.3],
 [5083, 17275, 0.7],
 [5083, 15931, 0.6],
 [5083, 16661, 0.5],
 [5083, 16775, 0.4],
 [5083, 15776, 0.3],
 

In [22]:
g.add_edges([(n, m) for n, m, _ in play_edges])
g.es['hours'] = [hours for _, _, hours in play_edges]

In [23]:
g.add_edges(purchase_edges)

In [24]:
edge_type = ['PLAYED' for _ in play_edges] + ['PURCHASED' for _ in purchase_edges]
g.es['edge_type'] = edge_type

For user 151603712, we can see that he has purchased 40 games.

In [25]:
user_id_ex = g.vs.select(steam_id_eq='151603712')
user_id_ex.indices

[1980]

In [26]:
user_id_ex = g.vs.select(steam_id_eq='151603712')[0].index

In [27]:
purchased_ex = g.es.select(_source_eq=user_id_ex, edge_type='PURCHASED')
print(len(list(purchased_ex)))

40


## Path based analytics in igraph

In [28]:
paths = g.get_all_simple_paths(user_id_ex, cutoff=3, mode='all')
print(paths[:10])

[[1980, 12772], [1980, 12772, 9], [1980, 12772, 9, 12655], [1980, 12772, 9, 12879], [1980, 12772, 9, 13004], [1980, 12772, 9, 13228], [1980, 12772, 9, 13415], [1980, 12772, 9, 13534], [1980, 12772, 9, 13701], [1980, 12772, 9, 13758]]


In [29]:
rec_game_ids = [path[3] for path in paths if len(path) == 4]

In [34]:
game_names = [g.vs[game_id]['steam_id'] for game_id in rec_game_ids]
game_names

['Patch testing for Chivalry',
 'Arma 2 Free',
 'BioShock Infinite',
 'Amnesia The Dark Descent',
 'Deadbreed',
 'Chivalry Medieval Warfare',
 'Half-Life 2 Lost Coast',
 'Age of Empires II HD Edition',
 'Half-Life 2 Episode Two',
 'Cosmophony',
 'Left 4 Dead',
 'Brawlhalla',
 'BioShock 2',
 'Unturned',
 'Half-Life 2 Episode One',
 'Nosgoth',
 'SpeedRunners',
 'Half-Life 2',
 'Half-Life Deathmatch Source',
 'Mitos.is The Game',
 'Zombies Monsters Robots',
 'Dirty Bomb',
 'Counter-Strike Global Offensive',
 'Half-Life 2 Deathmatch',
 'BioShock',
 'Team Fortress 2',
 'Left 4 Dead 2',
 'Counter-Strike',
 'Half-Life Source',
 'Ricochet',
 'Half-Life',
 'Counter-Strike Condition Zero',
 'Half-Life 2 Lost Coast',
 'Team Fortress Classic',
 'Half-Life 2 Episode Two',
 'Day of Defeat Source',
 'Counter-Strike Condition Zero Deleted Scenes',
 'Half-Life 2 Episode One',
 'Half-Life 2',
 'Half-Life Deathmatch Source',
 'Audiosurf',
 'Half-Life Opposing Force',
 'Deathmatch Classic',
 'Half-Life Bl

In [35]:
neighbors = g.neighbors(user_id_ex)
purchased_games = [g.vs[node_id]['steam_id'] for node_id in g.neighbors(user_id_ex)]
purchased_games

["Garry's Mod",
 "Garry's Mod",
 'Poly Bridge',
 'Poly Bridge',
 'BioShock Infinite',
 'BioShock Infinite',
 'The Elder Scrolls V Skyrim',
 'The Elder Scrolls V Skyrim',
 'Jazzpunk',
 'Jazzpunk',
 'SEGA Genesis & Mega Drive Classics',
 'SEGA Genesis & Mega Drive Classics',
 'Fallout New Vegas Dead Money',
 'Fallout 3 - Game of the Year Edition',
 'Fallout 3 - Game of the Year Edition',
 'Fallout 4',
 'Fallout 4',
 'The Banner Saga',
 'The Banner Saga',
 'Dead Island Epidemic',
 'Dead Island Epidemic',
 'Path of Exile',
 'Path of Exile',
 'Robocraft',
 'Robocraft',
 'Grand Theft Auto IV',
 'Grand Theft Auto IV',
 'Tomb Raider',
 'Tomb Raider',
 'Hitman Absolution',
 'Fallout New Vegas Honest Hearts',
 'HuniePop',
 'HuniePop',
 'Eldevin',
 'Eldevin',
 'Left 4 Dead',
 'Left 4 Dead',
 'The Elder Scrolls V Skyrim - Hearthfire',
 'BioShock 2',
 'Fallen Earth',
 'Realm of the Mad God',
 'Realm of the Mad God',
 'The Banner Saga - Mod Content',
 'Spore',
 'Spore',
 'Alan Wake',
 'The Elder Scr

In [36]:
game_names = [game for game in game_names if game not in purchased_games]
game_names

['Patch testing for Chivalry',
 'Arma 2 Free',
 'Amnesia The Dark Descent',
 'Deadbreed',
 'Chivalry Medieval Warfare',
 'Half-Life 2 Lost Coast',
 'Age of Empires II HD Edition',
 'Half-Life 2 Episode Two',
 'Cosmophony',
 'Brawlhalla',
 'Unturned',
 'Half-Life 2 Episode One',
 'Nosgoth',
 'SpeedRunners',
 'Half-Life 2',
 'Half-Life Deathmatch Source',
 'Mitos.is The Game',
 'Zombies Monsters Robots',
 'Dirty Bomb',
 'Counter-Strike Global Offensive',
 'Half-Life 2 Deathmatch',
 'Counter-Strike',
 'Half-Life Source',
 'Ricochet',
 'Half-Life',
 'Counter-Strike Condition Zero',
 'Half-Life 2 Lost Coast',
 'Team Fortress Classic',
 'Half-Life 2 Episode Two',
 'Day of Defeat Source',
 'Counter-Strike Condition Zero Deleted Scenes',
 'Half-Life 2 Episode One',
 'Half-Life 2',
 'Half-Life Deathmatch Source',
 'Audiosurf',
 'Half-Life Opposing Force',
 'Deathmatch Classic',
 'Half-Life Blue Shift',
 'Half-Life 2 Deathmatch',
 'Counter-Strike Source',
 'Day of Defeat',
 'Portal',
 'H1Z1 Test

In [33]:
from collections import Counter
game_frequency = Counter(game_names)
print(game_frequency)

Counter({'Counter-Strike Global Offensive': 5421, 'Unturned': 4155, 'Portal 2': 3998, 'Half-Life 2 Lost Coast': 3737, 'Portal': 3563, 'Borderlands 2': 3556, 'Counter-Strike Source': 3530, 'Terraria': 3455, 'Half-Life 2': 3398, "Sid Meier's Civilization V": 3085, 'Warframe': 3070, 'Half-Life 2 Deathmatch': 2805, 'PAYDAY 2': 2791, 'Metro 2033': 2668, 'Half-Life 2 Episode Two': 2593, 'Half-Life 2 Episode One': 2590, 'The Witcher 2 Assassins of Kings Enhanced Edition': 2493, 'Saints Row The Third': 2479, 'Skyrim High Resolution Texture Pack': 2343, 'Counter-Strike': 2269, 'Just Cause 2': 2202, 'War Thunder': 2173, 'Batman Arkham City GOTY': 2152, 'Heroes & Generals': 2102, 'Magicka': 2101, 'Dishonored': 2018, 'Torchlight II': 1967, 'PAYDAY The Heist': 1934, 'PlanetSide 2': 1927, 'Patch testing for Chivalry': 1910, 'Chivalry Medieval Warfare': 1910, "Mirror's Edge": 1897, 'Killing Floor': 1895, 'Dead Island': 1894, 'Amnesia The Dark Descent': 1880, 'Killing Floor Mod Defence Alliance 2': 18